In [ ]:
# TBD: create bash script
# To download kssl dataset
# pip install gdown install --upgrade --no-cache-dir gdown

# cd /root
# python
# import gdown
# id = "1qh8isMQIzhArsj3RNWJe6g0lx_cs7iBG" # MIRS KSSL v1 or "1SzQZqpbJuK3iCqjJYabuEobbgPXm3cHT" for v2
# gdown.download(id=id)
# quit()
# tar -xvf kssl.tar.gz

# paperspace only
#!pip install gdown 
#!pip install --upgrade gdown

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from fastai.vision.all import *
#from fastai.data.all import *
from tqdm import tqdm as tqdm

from sklearn.model_selection import train_test_split
import pandas as pd
import fastcore.all as fc
#from fastcore.basics import store_attr
from fastcore.foundation import *
from fastcore.xtras import *
from functools import partial

import timm

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from torchvision import transforms as T

#from spanda.data.transforms import (SpectraBlock, AnalytesBlock, SpectraTfm, 
#                                    AnalytesTfm, SNVTfm, RandWAvgTfm, NormalizeTfm, LogTfm)

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

use_cuda = torch.cuda.is_available()
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
#device = 'cpu'

In [ ]:
def meet_criteria(src_dir, analyte=725):
    has_spectrum = len(L(fname for fname in src_dir.ls() if fname.name != 'target.csv')) > 0
    y_in_range = (pd.read_csv(src_dir / 'target.csv', index_col='analyte').loc[analyte] > 0).value
    return has_spectrum and y_in_range

In [ ]:
paths = Path('../../_data/kssl-mirs-v2').ls(); paths

(#46305) [Path('../../_data/kssl-mirs-v2/180338'),Path('../../_data/kssl-mirs-v2/172221'),Path('../../_data/kssl-mirs-v2/177753'),Path('../../_data/kssl-mirs-v2/184798'),Path('../../_data/kssl-mirs-v2/53759'),Path('../../_data/kssl-mirs-v2/74947'),Path('../../_data/kssl-mirs-v2/176681'),Path('../../_data/kssl-mirs-v2/1855'),Path('../../_data/kssl-mirs-v2/175004'),Path('../../_data/kssl-mirs-v2/34499')...]

In [ ]:
paths_clean = L([path for path in tqdm(paths) if meet_criteria(path)]); paths_clean

100%|████████████████████████████████████████████████████████| 46305/46305 [00:32<00:00, 1413.22it/s]


(#44565) [Path('../../_data/kssl-mirs-v2/180338'),Path('../../_data/kssl-mirs-v2/172221'),Path('../../_data/kssl-mirs-v2/177753'),Path('../../_data/kssl-mirs-v2/184798'),Path('../../_data/kssl-mirs-v2/53759'),Path('../../_data/kssl-mirs-v2/74947'),Path('../../_data/kssl-mirs-v2/176681'),Path('../../_data/kssl-mirs-v2/1855'),Path('../../_data/kssl-mirs-v2/175004'),Path('../../_data/kssl-mirs-v2/34499')...]

In [ ]:
def get_x(src_dir, index='wavenumber'):
    fnames = L(fname for fname in src_dir.ls() if fname.name != 'target.csv')
    dfs = (pd.read_csv(fname, index_col=index) for fname in fnames)
    df = pd.concat(dfs, axis=1, ignore_index=False)
    return torch.tensor(df.values, dtype=torch.float).T

In [ ]:
get_x(Path('../../_data/kssl-mirs-v2/180338')).shape

torch.Size([4, 1700])

In [ ]:
def get_y(src_dir, index='analyte', analytes=None):
    df = pd.read_csv(src_dir / 'target.csv', index_col=index)
    if analytes:
        analytes = [analytes] if type(analytes) != 'list' else analytes
        df = df.loc[analytes]
    return torch.tensor(df.values, dtype=torch.float).squeeze()

In [ ]:
get_y(Path('../../_data/kssl-mirs-v2/180338'), analytes=725)

tensor(2.8077)

In [ ]:
class AvgTfm:
    def __init__(self, weighted=False):
        self.weighted = weighted
        
    def __call__(self, x): 
        if self.weighted:
            return self._weights(len(x))@x
        else:
            return torch.mean(x, dim=0, keepdim=True)
    
    def _weights(self, n):
        weights = torch.rand(n)
        return (weights/weights.sum()).unsqueeze(dim=0)

In [ ]:
class GADFTfm:
    def __init__(self, neg=True):
        self.neg = neg
        
    def rescale(self, x):
        m, M = x.min(), x.max()
        return ((x - M) + (x - m)) / (M - m) if self.neg else (x - m) / (M - m)

    def __call__(self, x):
        X, I = self.rescale(x), torch.ones((1, x.shape[1]))
        K = torch.sqrt(I - torch.square(X))
        return K.t()@X - X.t()@K

In [ ]:
class To3DTfm:
    def __init__(self, n=3440):
        n_channel = n // 3
        self.edge_side = ceil(sqrt(n_channel))
        self.padding = 3*self.edge_side**2 - n - 1
        
    def __call__(self, x):
        x = F.pad(x, (1, self.padding), mode='constant')
        return x.view(3, self.edge_side, self.edge_side)

In [ ]:
class ResizeTfm:
    def __init__(self, size=256):
        self.size = size
        
    def __call__(self, x):
        return T.Resize(self.size)(x.unsqueeze(0))

In [ ]:
class LogTfm:
    def __call__(self, ys):
        return torch.log10(ys)

In [ ]:
ResizeTfm()(GADFTfm()(get_x(Path('../../_data/kssl-mirs-v2/180338')))).shape

torch.Size([1, 256, 256])

In [ ]:
AvgTfm(weighted=True)(get_x(Path('../../_data/kssl-mirs-v2/180338')))

tensor([[0.1844, 0.1846, 0.1848,  ..., 1.5260, 1.5201, 1.5156]])

In [ ]:
transforms_x = T.Compose([GADFTfm(), ResizeTfm(224)])
#transforms_x = T.Compose([GADFTfm()])

In [ ]:
class NIRSDataset(Dataset):
    def __init__(self, src_dirs, get_x, get_y, transform=None, target_transform=None):
        #fc.store_attr()
        self.src_dirs = src_dirs
        self.transform = transform
        self.target_transform = target_transform
        self.get_x = get_x
        self.get_y = get_y

    def __len__(self):
        return len(self.src_dirs)

    def __getitem__(self, idx):
        x, y =  (getter(self.src_dirs[idx]) for getter in [self.get_x, self.get_y])
        if self.transform: 
            x = self.transform(x)
        if self.target_transform: 
            y = self.target_transform(y)
        return x, y

In [ ]:
paths_train, paths_valid = train_test_split(paths_clean, 
                                            test_size=0.2, 
                                            random_state=41)

In [ ]:
train_ds, valid_ds = [NIRSDataset(paths, 
                                  get_x=get_x,
                                  get_y=partial(get_y, analytes=725),
                                  transform=transforms_x,
                                  target_transform=LogTfm()
                                 )
                      for paths in [paths_train, paths_valid]] 

In [ ]:
def get_dls(train_ds, valid_ds, bs, **kwargs):
    Dataloaders = namedtuple('Dataloader', ['train', 'valid'])
    return Dataloaders(
        DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
        DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [ ]:
# prefetch_factor=4
n_workers = 8
dls = get_dls(train_ds, valid_ds, bs=32, num_workers=n_workers)

In [ ]:
def dl_time_per_epoch(t, n=40000, bs=32):
    dl_time = t*n/(bs*60)
    print(f'about {dl_time:.3} min per epoch')

In [ ]:
durations = []
n_batches = 10
i = 0
for x, y in dls.train:
    if i > 0: 
        duration = time.time() - start
        print(f'{duration} s per batch')
        durations.append(duration)
        
    if i > n_batches: break
    start = time.time()
    i += 1
    
print(f'on average: {np.array(durations).mean():.3} s per batch')
dl_time_per_epoch(np.array(durations).mean())

0.3301699161529541 s per batch
0.33040904998779297 s per batch
0.3276660442352295 s per batch
0.33043813705444336 s per batch
0.3216230869293213 s per batch
0.3367791175842285 s per batch
0.321397066116333 s per batch
0.32066774368286133 s per batch
0.3189711570739746 s per batch
0.32105183601379395 s per batch
0.33046793937683105 s per batch
on average: 0.326 s per batch
about 6.8 min per epoch


In [ ]:
next(iter(dls.train))[0].shape

torch.Size([32, 1, 224, 224])

In [ ]:
device

'mps'

In [ ]:
# .to_fp16()
learn = vision_learner(dls, 'resnet18', loss_func=MSELossFlat(), metrics=R2Score(), n_out=1, n_in=1)

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(3, 1e-2)

epoch,train_loss,valid_loss,r2_score,time


KeyboardInterrupt: 